<a href="https://colab.research.google.com/github/Jcnok/MELI_DATA_CHALLENGE_2021/blob/main/Uniformbaseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Criando um modelo base

In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from numpy import cumsum
import os

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
#Acesso ao diretório de estudo do drive
os.chdir('/content/drive/MyDrive/Colab_Notebooks/comp_ML')

In [5]:
train = pd.read_csv('treino.csv') #Carregando o conjunto de amostra

In [6]:
train.date.min() # menor data do conjunto

'2021-02-01'

In [7]:
train.date.max() # Maior data do conjunto

'2021-03-31'

In [8]:
data_val = train.loc[(train.date >'2021-03-01')] # Separando os ultimos 30 dias para validadação (conjunto de validação)
data_val

,Unnamed: 0,sku,date,sold_quantity,current_price,currency,listing_type,shipping_logistic_type,shipping_payment,minutes_active
29,2912,78385,2021-03-02,3,53.90,REA,premium,fulfillment,paid_shipping,1440.0
30,2913,78385,2021-03-03,1,53.90,REA,premium,fulfillment,paid_shipping,1440.0
31,2914,78385,2021-03-04,4,53.90,REA,premium,fulfillment,paid_shipping,1440.0
32,2915,78385,2021-03-05,4,53.90,REA,premium,fulfillment,paid_shipping,1440.0
33,2916,78385,2021-03-06,2,53.90,REA,premium,fulfillment,paid_shipping,1440.0
...,...,...,...,...,...,...,...,...,...,...
1116,165355,352804,2021-03-27,0,26.96,REA,premium,fulfillment,paid_shipping,0.0
1117,165356,352804,2021-03-28,0,26.96,REA,premium,fulfillment,paid_shipping,0.0
1118,165357,352804,2021-03-29,0,26.96,REA,premium,fulfillment,paid_shipping,0.0
1119,165358,352804,2021-03-30,0,26.96,REA,premium,fulfillment,paid_shipping,0.0


In [9]:
data_train = train.loc[(train.date <= '2021-03-01')] # Usando o resto para o conjunto de treino
data_train

,Unnamed: 0,sku,date,sold_quantity,current_price,currency,listing_type,shipping_logistic_type,shipping_payment,minutes_active
0,2883,78385,2021-02-01,3,53.90,REA,premium,fulfillment,paid_shipping,1440.000000
1,2884,78385,2021-02-02,2,53.90,REA,premium,fulfillment,paid_shipping,1440.000000
2,2885,78385,2021-02-03,1,53.90,REA,premium,fulfillment,paid_shipping,1440.000000
3,2886,78385,2021-02-04,3,53.90,REA,premium,fulfillment,paid_shipping,1440.000000
4,2887,78385,2021-02-05,1,53.90,REA,premium,fulfillment,paid_shipping,1440.000000
...,...,...,...,...,...,...,...,...,...,...
1086,165325,352804,2021-02-25,0,26.96,REA,premium,fulfillment,paid_shipping,1440.000000
1087,165326,352804,2021-02-26,0,26.96,REA,premium,fulfillment,paid_shipping,1440.000000
1088,165327,352804,2021-02-27,1,26.96,REA,premium,fulfillment,paid_shipping,1440.000000
1089,165328,352804,2021-02-28,0,26.96,REA,premium,fulfillment,paid_shipping,1440.000000


In [10]:
#Função para preparar o conjunto de validação
def create_validation_set(dataset):
  #Definindo a semente
  np.random.seed(42)
  # realizando a triagem
  print('Sorting records...')
  # variável ordenada por sku e data
  temp_pd = dataset.loc[:, ['sku','date','sold_quantity']].sort_values(['sku','date'])
  #realizando o agrupamento da quantidade vendida
  print('Grouping quantity...')
  #Cria uma var tipo dicionario com a lista de todas as quantidades vendidas por sku
  temp_dict = temp_pd.groupby('sku').agg({'sold_quantity':lambda x: [i for i in x]})['sold_quantity'].to_dict()
  #cria uma lista em branco
  result = []
  # percorrendo todos os items do dicionário por id e lista
  for idx, list_quantity in tqdm(temp_dict.items(), desc='Making targets...'):
    #variável que recebe o valor da soma acumulada da lista.
    cumsum = np.array(list_quantity).cumsum()
    #[0,0,0,1,2,2,2,5]
    
    
    stock_target = 0
    #se o ultimo valor da lista for maior que 0 e o tamanho da lista for igual a 30 faça:
    if cumsum[-1] > 0 and (len(cumsum) == 30 or len(cumsum) == 29):
      
      #Faça enquanto a var for igual a 0
      while stock_target == 0:
        #var stock_target recebe um valor aleatório de cumsum
        stock_target = np.random.choice(cumsum)
       
      #recebe o primeiro indice onde cumsum é igual ao stock_target e acrecenta 1 pois o indice em python inicia por 0
      day_to_stockout = np.argwhere(cumsum == stock_target).min() + 1
      #adiciona o resultado a lista.
      result.append({'sku':idx, 'target_stock':stock_target, 'inventory_days':day_to_stockout})
    #elif cumsum[-1] == 0:#senão existir algum registro sem estoque e sem vendas / sem o elif ele não cria registros sem vendas no mês
    #  day_to_stockout = len(cumsum) # adiciona o tamanho da var cumsum, pois são os dias em que não houve venda alguma
    #  result.append({'sku':idx, 'target_stock':stock_target, 'inventory_days':day_to_stockout})

  return result

#val_dataset = create_validation_set(data_val)

In [12]:
#Criando o dicionario de validação
val_dataset = create_validation_set(data_val)
val_dataset

Sorting records...
Grouping quantity...


Making targets...: 100%|██████████| 19/19 [00:00<00:00, 4824.25it/s]


[{'inventory_days': 7, 'sku': 60683, 'target_stock': 7},
 {'inventory_days': 7, 'sku': 74185, 'target_stock': 3},
 {'inventory_days': 29, 'sku': 78385, 'target_stock': 40},
 {'inventory_days': 5, 'sku': 136815, 'target_stock': 4},
 {'inventory_days': 11, 'sku': 158820, 'target_stock': 30},
 {'inventory_days': 5, 'sku': 182644, 'target_stock': 1},
 {'inventory_days': 29, 'sku': 183080, 'target_stock': 28},
 {'inventory_days': 21, 'sku': 302930, 'target_stock': 3},
 {'inventory_days': 6, 'sku': 347586, 'target_stock': 5},
 {'inventory_days': 25, 'sku': 358099, 'target_stock': 6},
 {'inventory_days': 18, 'sku': 414978, 'target_stock': 6},
 {'inventory_days': 16, 'sku': 546617, 'target_stock': 61},
 {'inventory_days': 9, 'sku': 590037, 'target_stock': 3},
 {'inventory_days': 4, 'sku': 596056, 'target_stock': 4},
 {'inventory_days': 19, 'sku': 619239, 'target_stock': 3},
 {'inventory_days': 6, 'sku': 621261, 'target_stock': 1}]

# Criando o modelo base de distribuição uniforme

In [13]:
#Dicionario de validação - resultado do def create_validation_set com o conjunto do mes 3
val_dataset

[{'inventory_days': 7, 'sku': 60683, 'target_stock': 7},
 {'inventory_days': 7, 'sku': 74185, 'target_stock': 3},
 {'inventory_days': 29, 'sku': 78385, 'target_stock': 40},
 {'inventory_days': 5, 'sku': 136815, 'target_stock': 4},
 {'inventory_days': 11, 'sku': 158820, 'target_stock': 30},
 {'inventory_days': 5, 'sku': 182644, 'target_stock': 1},
 {'inventory_days': 29, 'sku': 183080, 'target_stock': 28},
 {'inventory_days': 21, 'sku': 302930, 'target_stock': 3},
 {'inventory_days': 6, 'sku': 347586, 'target_stock': 5},
 {'inventory_days': 25, 'sku': 358099, 'target_stock': 6},
 {'inventory_days': 18, 'sku': 414978, 'target_stock': 6},
 {'inventory_days': 16, 'sku': 546617, 'target_stock': 61},
 {'inventory_days': 9, 'sku': 590037, 'target_stock': 3},
 {'inventory_days': 4, 'sku': 596056, 'target_stock': 4},
 {'inventory_days': 19, 'sku': 619239, 'target_stock': 3},
 {'inventory_days': 6, 'sku': 621261, 'target_stock': 1}]

In [14]:
#dias para predição
days_to_predict = 30

In [15]:
#tamanho do dicionario de validação.
len(val_dataset)

16

In [16]:
#preenchendo a lista com valor = 1 com 30 repetições
np.ones(days_to_predict)

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [17]:
# Divido o valor 1/30 e arredondo para 4 casas decimais.
(np.ones(days_to_predict)/days_to_predict).round(4).tolist()[:5]

[0.0333, 0.0333, 0.0333, 0.0333, 0.0333]

In [18]:
# cria um array de dimensão 30 com valores = 1 e divide por 30 com 4 casas decimais depois repete a lista pelo tamanho de len(val_dataset) e ao final multiplica pelo tamanho de val_dataset
y_pred_uniform = [(np.ones(days_to_predict)/days_to_predict).round(4).tolist()] * len(val_dataset)
# basicamente tempo n(val_dataset) com days_to_predict valores iguais a 1/30 round 4.
y_pred_uniform[:1] 

[[0.0333,
  0.0333,
  0.0333,
  0.0333,
  0.0333,
  0.0333,
  0.0333,
  0.0333,
  0.0333,
  0.0333,
  0.0333,
  0.0333,
  0.0333,
  0.0333,
  0.0333,
  0.0333,
  0.0333,
  0.0333,
  0.0333,
  0.0333,
  0.0333,
  0.0333,
  0.0333,
  0.0333,
  0.0333,
  0.0333,
  0.0333,
  0.0333,
  0.0333,
  0.0333]]

In [19]:
range(1,days_to_predict+1) # Cria o nome das colunas de 1 a 31 que são a repesentação dos dias

range(1, 31)

In [21]:
# Convertendo a variável em df pandas, esse será nossa variável de predição.
base_model = pd.DataFrame(y_pred_uniform, columns=range(1,days_to_predict+1))
# aqui temos nossa base com o tamanho e com os dias já divididos em partes iguais.

In [22]:
#Salvando o modelo base no formato correto para submissão!
pd.DataFrame(base_model).round(4).to_csv("base_model.csv.gz", header=False, index=False, compression="gzip")

In [23]:
#verificando se o arquivo foi salvo corretamente
pd.read_csv('base_model.csv.gz', header=None).head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
0,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333
1,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333
2,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333
3,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333
4,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333


* Temos nosso modelo base uniforme preparado agora vamos criar uma função para calcular o RPS score.


# Função Ranked Probability Score (RPS):

In [24]:
def ranked_probability_score(y_true, y_pred):
  """
  Input
      y_true: np.array of shape 30.
      y_pred: np.array of shape 30.
  """
  #subtração de coluna por coluna entre os conjuntos de validação - predição 
  #Ele o resultado ao quadrado para remover valores negativos;
  #Soma de todos os valores seguido da média
  #Resultado do rps.
  return ((y_true.cumsum(axis= 1) - y_pred.cumsum(axis= 1))**2).sum(axis= 1).mean()

def scoring_funcion(y_true, y_pred):
  """
  Input
      y_true: List of ints of shape Nx1. Contain the target_stock
      y_pred: List of float of shape Nx30. Contain the prob for each day
  """
  #y_val de validação que nada mais é que aleatóriamente o tempo que levou em dias para meu estoque se esgotar.
  y_true = np.array(y_true)
  #y_pred array com o tamnho dos skus e dos dias para com os valores divididos de forma homogenea.
  y_pred = np.array(y_pred)
  #cria n arrays(y_pred) com valores igual 0 nesse caso 16x30 tipo float
  y_true_one_hot = np.zeros_like(y_pred, dtype = np.float)
  #em um range de tamanho y_true(0,16) na posição onde o valor de y_true-1 adicione o valor 1
  #basicamente aqui ele faz uma marcação com 1 que corresponde ao número mínimo de dias que o produto ficou em estoque
  y_true_one_hot[range(len(y_true)), y_true - 1] = 1
  return ranked_probability_score(y_true_one_hot, y_pred)

In [25]:
# Variável que recebe o valor de inventory_days do dic val_dataset.
y_true_val = [x['inventory_days'] for x in val_dataset]
y_true_val

[7, 7, 29, 5, 11, 5, 29, 21, 6, 25, 18, 16, 9, 4, 19, 6]

In [26]:
#chamando a função RPS com a validação e nossa predição de distribuição uniforme
uniform_score = scoring_funcion(y_true_val, y_pred_uniform)
print('Uniform model got a validation RPS of:', uniform_score)
#no quadro da leaderbord obteve uma pontuação de 5,07 e na nossa amostragem obteve 5.09
# 5.7848 resultado ao incluir registros com 0 de vendas

Uniform model got a validation RPS of: 5.09074245
